<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [123]:
# TODO - Words, words, mere words, no matter from the heart.

from urllib.request import urlopen

res = urlopen("https://www.gutenberg.org/files/100/100-0.txt")
if res.status == 200:
    text = res.read().decode("utf-8")

In [60]:
chars = list(set(text))

i_to_c = {i: c for i, c in enumerate(chars)}
c_to_i = {c: i for i, c in enumerate(chars)}

encoded = [c_to_i[c] for c in text]
sequences = []
next_char = []

In [61]:
maxlen = 30
step = 5

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])

    

In [63]:
import numpy as np

X = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        X[i, t, char] = 1
    y[i, next_char[i]] = 1

X.shape

(1037905, 30, 63)

In [64]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

max_features = 20000

model = Sequential()
model.add(LSTM(len(chars), input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam")

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 63)                32004     
_________________________________________________________________
dense_8 (Dense)              (None, 63)                4032      
Total params: 36,036
Trainable params: 36,036
Non-trainable params: 0
_________________________________________________________________


In [65]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, c_to_i[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = i_to_c[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [66]:
import random, sys

model.fit(X, y, batch_size=128, epochs=10, callbacks=[print_callback],)

Train on 1037905 samples
Epoch 1/10
1037824/1037905 [============================>.] - ETA: 0s - loss: 2.2824
----- Generating text after Epoch: 0
----- Generating with seed: "ll never hold that man my frie"
ll never hold that man my fries  JUIVANGUSS Orothim of to fors hish What dutsons you thy some for thourd a on andies far scoode    wirguod thim     Bure bot be fif goe noY hil shouny teres    ITht ang cnakice dringRangure is eriss deow he home lives and gere coth or hat      ang somandOO Castary Thas Fomest yiu gureeOThat the of bodest thig baghy the afe he of feencbeld anvor wemaruing    Wiylat be thie Voll dle gigeende thou 
1037905/1037905 [==============================] - 313s 302us/sample - loss: 2.2824
Epoch 2/10
1037824/1037905 [============================>.] - ETA: 0s - loss: 2.0014
----- Generating text after Epoch: 1
----- Generating with seed: "d room enoughWhen there is in "
d room enoughWhen there is in mansirk wrange    Andreon Bensade weld lasse SDOUK These workned

In [138]:

res = urlopen("https://www.gutenberg.org/files/100/100-0.txt")
if res.status == 200:
    text = res.read().decode("utf-8")
    
section_titles = """

               THE SONNETS

               ALL’S WELL THAT ENDS WELL

               THE TRAGEDY OF ANTONY AND CLEOPATRA

               AS YOU LIKE IT

               THE COMEDY OF ERRORS

               THE TRAGEDY OF CORIOLANUS

               CYMBELINE

               THE TRAGEDY OF HAMLET, PRINCE OF DENMARK

               THE FIRST PART OF KING HENRY THE FOURTH

               THE SECOND PART OF KING HENRY THE FOURTH

               THE LIFE OF KING HENRY THE FIFTH

               THE FIRST PART OF HENRY THE SIXTH

               THE SECOND PART OF KING HENRY THE SIXTH

               THE THIRD PART OF KING HENRY THE SIXTH

               KING HENRY THE EIGHTH

               KING JOHN

               THE TRAGEDY OF JULIUS CAESAR

               THE TRAGEDY OF KING LEAR

               LOVE’S LABOUR’S LOST

               THE TRAGEDY OF MACBETH

               MEASURE FOR MEASURE

               THE MERCHANT OF VENICE

               THE MERRY WIVES OF WINDSOR

               A MIDSUMMER NIGHT’S DREAM

               MUCH ADO ABOUT NOTHING

               THE TRAGEDY OF OTHELLO, MOOR OF VENICE

               PERICLES, PRINCE OF TYRE

               KING RICHARD THE SECOND

               KING RICHARD THE THIRD

               THE TRAGEDY OF ROMEO AND JULIET

               THE TAMING OF THE SHREW

               THE TEMPEST

               THE LIFE OF TIMON OF ATHENS

               THE TRAGEDY OF TITUS ANDRONICUS

               THE HISTORY OF TROILUS AND CRESSIDA

               TWELFTH NIGHT; OR, WHAT YOU WILL

               THE TWO GENTLEMEN OF VERONA

               THE TWO NOBLE KINSMEN

               THE WINTER’S TALE

               A LOVER’S COMPLAINT

               THE PASSIONATE PILGRIM

               THE PHOENIX AND THE TURTLE

               THE RAPE OF LUCRECE

               VENUS AND ADONIS
"""

titles = [title.strip() for title in section_titles.split("\n") if title]

In [139]:
titles = [
    r"(((\n)|(\r)){3,}" + title + "((\n)|(\r)){3,})"
    for title in titles
]
print(titles[:2])

['(((\\n)|(\\r)){3,}THE SONNETS((\n)|(\r)){3,})', '(((\\n)|(\\r)){3,}ALL’S WELL THAT ENDS WELL((\n)|(\r)){3,})']


In [140]:
titles_re = re.compile(
    "|".join(titles)
)

In [142]:
res = titles_re.split(text)

In [175]:
filtered = [x for x in res if x][1:]
thresh = 10
while len(filtered) > 44:
    filtered = [x for x in filtered if x and len(x) > thresh]
    thresh += 1

In [176]:
title_to_contents = {
    title: contents
    for title, contents in zip(titles, filtered)
}

In [193]:
title_to_contents2 = {}
for k, v in title_to_contents.items():
    
    title_to_contents2[k[16:-14]] = v
    print(k[16:-14], v.strip()[:100])
    
    print()
    print()
    

THE SONNETS 1

From fairest creatures we desire increase,
That thereby beauty’s rose might never die,
But as


ALL’S WELL THAT ENDS WELL Contents

ACT I
Scene I. Rossillon. A room in the Countess’s palace.
Scene II. Paris. A room in 


THE TRAGEDY OF ANTONY AND CLEOPATRA DRAMATIS PERSONAE

  MARK ANTONY,         Triumvirs
  OCTAVIUS CAESAR,         "
  M. AEMILIUS L


AS YOU LIKE IT DRAMATIS PERSONAE.

  DUKE, living in exile
  FREDERICK, his brother, and usurper of his dominion


THE COMEDY OF ERRORS Contents

ACT I
Scene I. A hall in the Duke’s palace.
Scene II. A public place.


ACT II
Sce


THE TRAGEDY OF CORIOLANUS Dramatis Personae

  CAIUS MARCIUS, afterwards CAIUS MARCIUS CORIOLANUS

    Generals against th


CYMBELINE Contents

ACT I
Scene I. Britain. The garden of Cymbeline’s palace.
Scene II. The same.
Scene I


THE TRAGEDY OF HAMLET, PRINCE OF DENMARK THE TRAGEDY OF HAMLET, PRINCE OF DENMARK


THE FIRST PART OF KING HENRY THE FOURTH Contents

ACT I
Scene I. Elsinore. A plat

In [190]:
max_features = 20000

model = Sequential()
model.add(LSTM(len(chars), input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam")

model.summary()


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 63)                32004     
_________________________________________________________________
dense_10 (Dense)             (None, 63)                4032      
Total params: 36,036
Trainable params: 36,036
Non-trainable params: 0
_________________________________________________________________


In [194]:
sonnets = title_to_contents["THE SONNETS"]
sonnets

'                    1\r\n\r\nFrom fairest creatures we desire increase,\r\nThat thereby beauty’s rose might never die,\r\nBut as the riper should by time decease,\r\nHis tender heir might bear his memory:\r\nBut thou contracted to thine own bright eyes,\r\nFeed’st thy light’s flame with self-substantial fuel,\r\nMaking a famine where abundance lies,\r\nThy self thy foe, to thy sweet self too cruel:\r\nThou that art now the world’s fresh ornament,\r\nAnd only herald to the gaudy spring,\r\nWithin thine own bud buriest thy content,\r\nAnd, tender churl, mak’st waste in niggarding:\r\n  Pity the world, or else this glutton be,\r\n  To eat the world’s due, by the grave and thee.\r\n\r\n\r\n                    2\r\n\r\nWhen forty winters shall besiege thy brow,\r\nAnd dig deep trenches in thy beauty’s field,\r\nThy youth’s proud livery so gazed on now,\r\nWill be a tattered weed of small worth held:\r\nThen being asked, where all thy beauty lies,\r\nWhere all the treasure of thy lusty days

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN